In [12]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema import Document
os.environ["OPENAI_API_KEY"] = "..."

openai_embedding=OpenAIEmbeddings(model = 'text-embedding-3-small')

In [2]:
def load_and_split_pdf(file_path):
    loader = PyPDFLoader(file_path)
    return loader.load_and_split()

In [3]:
def create_vector_store(_docs):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(_docs)
    persist_directory = "./chroma_db"
    vectorstore = Chroma.from_documents(
        split_docs, 
        OpenAIEmbeddings(model='text-embedding-3-small'),
        persist_directory=persist_directory
    )
    return vectorstore

In [4]:
def add_to_vector_store(new_docs, persist_directory="./chroma_db"):
    # 1. 기존 벡터 저장소 불러오기
    vectorstore = Chroma(
        persist_directory=persist_directory,
        embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
    )

    # 2. 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_docs = text_splitter.split_documents(new_docs)

    # 3. 문서 추가
    vectorstore.add_documents(split_docs)

    return vectorstore

In [7]:
path='../하지환교수님'
pl=os.listdir(path)

In [8]:
mk=load_and_split_pdf(os.path.join(path, pl[0]))
create_vector_store(mk)

In [9]:
for i in pl[1:]:
    ad=load_and_split_pdf(os.path.join(path, i))
    add_to_vector_store(ad)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9000\3363452400.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [10]:
# load from disk
db3 = Chroma(
    persist_directory="./chroma_db",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

query = "HyungBin Moon's 2021 paper?"
result = db3.similarity_search(query)
print(result[0].page_content)

Manuscript received 12 December 2020; revised 16 May 2022; accepted 8 July
2022. Date of publication 18 July 2022; date of current version 3 April 2023.
This work was supported by the Institute of Information & Communications
Technology Planning & Evaluation (IITP), funded by the Korean Government,
MSIT, under Grant IITP-2017-0-00477, (SW starlab) Research and develop-
ment of the high performance in-memory distributed DBMS based on ﬂash
memory storage in an IoT environment.
(Corresponding author: Sanghyun Park.)
Digital Object Identiﬁer no. 10.1109/TCBB.2022.3191972
IEEE/ACM TRANSACTIONS ON COMPUTATIONAL BIOLOGY AND BIOINFORMATICS, VOL. 20, NO. 2, MARCH/APRIL 2023 1257
This work is licensed under a Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 License. For more information, see https://creativecommons.org/licenses/by-nc-nd/4.0/


In [11]:
db3 = Chroma(
    persist_directory="./chroma_db",
    embedding_function=OpenAIEmbeddings(model="text-embedding-3-small")
)

query = "explain jiwhan Ha's 2020 paper"
docs = db3.similarity_search(query, k=15)

formatted_docs = [Document(page_content=doc.page_content) for doc in docs]

prompt_template = """
다음은 논문 일부입니다. 이 내용을 바탕으로 석사 논문 내용을 연구대상 및 방법, 연구방법, 연구결과, 제언을 핵심적으로 요약해 주세요.
논문 내용:
{context}
요약:
"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["context"])
llm = ChatOpenAI(model="gpt-4o")
chain = LLMChain(llm=llm, prompt=PROMPT)
context_text = "\n\n".join([doc.page_content for doc in formatted_docs])
summary = chain.run(context=context_text)
print(summary, "\n")

C:\Users\Admin\AppData\Local\Temp\ipykernel_9000\3330118453.py:19: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4o")
C:\Users\Admin\AppData\Local\Temp\ipykernel_9000\3330118453.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=PROMPT)
C:\Users\Admin\AppData\Local\Temp\ipykernel_9000\3330118453.py:22: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = chain.run(context=context_text)


**연구대상 및 방법:**
이 연구는 miRNA(마이크로 리보핵산)와 질병 간의 연관성을 예측하는 것을 목표로 합니다. 연구 대상은 다양한 생물학적 데이터를 활용하여 miRNA-질병 연관성을 분석하는 것이며, 이를 통해 miRNA가 질병에 미치는 영향을 이해하고자 합니다.

**연구방법:**
연구에서는 기계 학습과 데이터 마이닝 기법을 사용하여 miRNA-질병 연관성을 예측합니다. 구체적으로, 그래프 컨볼루션 네트워크(Graph Convolutional Network, GCN)와 신경 협업 필터링(Neural Collaborative Filtering)을 결합하여 miRNA와 질병 간의 연관성을 모델링합니다. 또한, 네트워크 거리 분석 및 행렬 분해(Matrix Factorization) 기법을 활용하여 질병 유사성 제약을 고려한 모델을 설계합니다.

**연구결과:**
연구 결과, 제안된 방법론이 miRNA-질병 연관성을 예측하는 데 있어 높은 성능을 보였습니다. Kaplan-Meier 생존 분석을 통해 특정 miRNA(hsa-miR-148a, hsa-miR-133b 등)가 유방암 환자의 생존율에 미치는 영향을 시각화하였으며, IMIPMF를 사용하여 학습된 잠재 벡터를 통해 miRNA의 역할을 효과적으로 분석할 수 있음을 보였습니다.

**제언:**
본 연구는 miRNA-질병 연관성 예측의 정확성을 높이기 위해 다양한 생물학적 데이터의 의미 있는 특징을 추출할 필요성을 제안합니다. 향후 연구에서는 새로운 기계 학습 기법과 데이터 통합 방법을 개발하여 miRNA와 질병 간의 복잡한 상호작용을 더욱 정교하게 분석할 것을 권장합니다. 이러한 접근은 궁극적으로 개인화된 의학 및 정밀 의학 발전에 기여할 수 있을 것입니다. 

